In [ ]:
# sql 환경 세팅

%load_ext sql
%sql postgresql://[username]:[passowrd]@[host]:[port]/[database]

In [ ]:
import pandas as pd
import re

In [ ]:
# 크롤링한 데이터 로드
data = pd.read_csv('./hotdeal-info.csv')

In [ ]:
data

In [ ]:
# Mainboard 키워드 선정

mb_name_list = []
for title in data['title']:  # 제목에서
    mb_name = re.findall('[ABXHZabxhz]\d\d0M', title)  # 알파벳 숫자 숫자 0 M 꼴 텍스트 검색
    mb_name_list += mb_name
mb_name_list = list(set(mb_name_list))
mb_name_list

In [ ]:
# PWR 키워드 선정

pwr_name_list = []
for title in data['title']:  # 제목에서
    pwr_name = re.findall('[0-9]?[0-9]?[0-9]0W', title)  # '숫자 숫자 숫자 0 W' 꼴 텍스트 검색
    pwr_name_list += pwr_name
pwr_name_list = sorted(list(set(pwr_name_list)), key=lambda x:int(x[:-1]))
pwr_name_list = pwr_name_list[11:] # 200W 이상인 제품 추출 - 200W 이하인 경우 배터리, 충전기 등의 제품 포함 검색
pwr_name_list

In [ ]:
import csv

with open("mb_keyword.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(mb_name_list)

with open("pwr_keyword.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(pwr_name_list)

In [ ]:
%%sql

-- sample_data 스키마 생성
CREATE SCHEMA IF NOT EXISTS sample_data;

In [ ]:
%%sql

-- 원본 데이터 hotdeal_info 테이블 생성
CREATE TABLE sample_data.hotdeal_info (
  title varchar(255),
  created_at timestamp,
  price varchar(255),
  views integer,
  votes integer,
  shop_type varchar(255)
)

In [ ]:
%%sql

-- 아래 쿼리에서 aws_iam_role의 값은 각자 Redshift cluster에 지정한 S3 읽기 권한 ROLE의 ARN을 지정
COPY sample_data.hotdeal_info
FROM [s3 url]
credentials 'aws_iam_role=[iam arn]'
delimiter ',' dateformat 'auto' timeformat 'auto' IGNOREHEADER 1 removequotes;

In [ ]:
%%sql

-- MAIN BOARD 키워드 테이블
CREATE TABLE IF NOT EXISTS sample_data.mainboard_item (
   product_name varchar(5)
);

-- POWER SUPPLY 키워드 테이블
CREATE TABLE IF NOT EXISTS sample_data.power_supply_item (
   product_name varchar(10)
);

In [ ]:
%%sql

-- MAIN BOARD 키워드 입력
-- mb_keyword.csv파일에서 추출된 결과 값을 정리해서 입력
INSERT INTO sample_data.mainboard_item (product_name)
VALUES
 ('B450'),
 ('X370'),
 ('Z690'),
 ('Z590'),
 ('B660'),
 ('Z370'),
 ('B350'),
 ('B250'),
 ('A520'),
 ('B760'),
 ('Z270'),
 ('H310'),
 ('H510'),
 ('B550'),
 ('B460'),
 ('B360'),
 ('B650'),
 ('H410'),
 ('Z490'),
 ('Z390'),
 ('B560'),
 ('A620'),
 ('A320'),
 ('H370'),
 ('H570'),
 ('H610');

In [ ]:
%%sql

-- POWER SUPPLY 키워드 입력
-- pwr_keyword.csv파일에서 추출된 결과 값을 정리해서 입력
INSERT INTO sample_data.power_supply_item (product_name)
VALUES
 ('750W'),
 ('800W'),
 ('420W'),
 ('310W'),
 ('1600W'),
 ('1670W'),
 ('850W'),
 ('1300W'),
 ('1560W'),
 ('500W'),
 ('1000W'),
 ('1780W'),
 ('270W'),
 ('210W'),
 ('600W'),
 ('900W'),
 ('330W'),
 ('700W'),
 ('550W'),
 ('2160W'),
 ('1650W'),
 ('1050W'),
 ('400W'),
 ('1250W'),
 ('200W'),
 ('240W'),
 ('3700W'),
 ('3680W'),
 ('5310W'),
 ('890W'),
 ('9500W'),
 ('810W'),
 ('3300W'),
 ('710W'),
 ('220W'),
 ('2030W'),
 ('630W'),
 ('5500W'),
 ('770W'),
 ('650W'),
 ('1200W')

In [ ]:
%%sql

-- MAIN BOARD
SELECT A.title
     , A.created_at
     , REGEXP_REPLACE(price,'[^0-9.]+','') AS price
     --, REGEXP_REPLACE(price,'[^a-zA-Z]','') currency
     , A.shop_type
     , A.votes
     , A.views
     , B.product_name AS product
     , 'mainboard' AS components_of_computer
  FROM sample_data.hotdeal_info A
  JOIN sample_data.mainboard_item B
    ON TRIM(' ' FROM UPPER(A.title)) LIKE ('%' || UPPER(B.product_name) || '%') -- 대문자로 치환해서 product_name 값이 들어있는 단어 추출
;

In [ ]:
%%sql

-- POWER SUPPLY 전처리 생성
SELECT A.title
     , A.created_at
     , REGEXP_REPLACE(price,'[^0-9.]+','') AS price
     --, REGEXP_REPLACE(price,'[^a-zA-Z]','') currency
     , A.shop_type
     , A.votes
     , A.views
     , B.product_name AS product
     , 'power supply' AS components_of_computer
  FROM sample_data.hotdeal_info A
  JOIN sample_data.power_supply_item B
    ON TRIM(' ' FROM UPPER(A.title)) LIKE ('% ' || B.product_name || ' %') -- 대문자 치환 후 product_name 값이 들어있는 단어 추출
;